In [1]:
import pandas as pd 
import numpy as np 

import pyodbc
from google.cloud import bigquery

from datetime import datetime

In [2]:
# Pull all code volumes by physician, by year

In [3]:
bq_conn = bigquery.Client()

In [4]:
ssms_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DHCSQLP02;DATABASE=ANALYST_WORK;Trusted_Connection=yes;')

#### Procedures - Medicare

In [5]:

### MEDICARE INPATIENT PROCEDURES ###

def pull_medicare_inpatient_procs(year, qtrly=0):
    
    if qtrly == 1:
        q = '_q'
        q2 = '_QTRLY'
    else:
        q = ''
        q2 = ''
    
    med_inp_pr_sql = """
    SELECT  {yr} AS CLAIM_YEAR
        ,   PROCEDURE_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END AS AGE_RANGE 
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END AS GENDER 
        ,   A.PHYSICIAN_NPI
        ,   'Medicare' AS CLAIMS_SOURCE 
        ,   COUNT(*) AS NUM_PROCEDURES
        ,   COUNT(DISTINCT C.DESY_SORT_KEY) AS NUM_PATIENTS 
    FROM    (
            SELECT  DISTINCT A.DESY_SORT_KEY 
                ,   A.CLM_THRU_DT 
                ,   B.CLM_PRCDR_CD_ORIG AS PROCEDURE_CODE 
                ,   COALESCE(A.OP_PHYSN_NPI, A.AT_PHYSN_NPI) AS PHYSICIAN_NPI
            FROM    MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}{q} A 
            JOIN    MEDICARE_SAF_{yr}.dbo.INPATIENT_CLM_PRCDR{q2} B 
            ON      A.CLAIM_NO = B.CLAIM_NO 
            ) A
    JOIN    MEDICARE_SAF_{yr}.dbo.CLMS_DENOMINATOR_MEDICARE{q2} C 
    ON      A.DESY_SORT_KEY = C.DESY_SORT_KEY
    GROUP   BY PROCEDURE_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END
        ,   A.PHYSICIAN_NPI
    """.format(yr=year, q=q, q2=q2)

    med_inp_pr_df = pd.read_sql(med_inp_pr_sql, ssms_conn)
    
    return med_inp_pr_df 

In [6]:
for year in[2018, 2019]:
    if year==2019:
        qtrly=1
    else:
        qtrly=0
    print('Running Medicare Inpatient for {yr}'.format(yr=year))
    med_inp_pr_result = pull_medicare_inpatient_procs(year, qtrly)
    
    print('Medicare Inpatient for {yr} complete.'.format(yr=year))
    dataframe = med_inp_pr_result
    
    print('Loading Medicare Inpatient for {yr} to BQ.'.format(yr=year))
    tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_PROCEDURES_MEDICARE_YEARLY'
    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    print(job.result())

Running Medicare Inpatient for 2018
Medicare Inpatient for 2018 complete.
Loading Medicare Inpatient for 2018 to BQ.


/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'PROCEDURE_CODE'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Running Medicare Inpatient for 2019
Medicare Inpatient for 2019 complete.
Loading Medicare Inpatient for 2019 to BQ.


In [5]:
### MEDICARE OUTPATIENT PROCEDURES ###

def pull_medicare_outpatient_procs(year, qtrly, cd):
    
    if qtrly == 1:
        q = '_q'
        q2 = '_QTRLY'
    else:
        q = ''
        q2 = ''
        
    med_out_pr_sql = """
    SELECT   {yr} AS CLAIM_YEAR
        ,   PROCEDURE_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END AS AGE_RANGE 
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END AS GENDER 
        ,   A.PHYSICIAN_NPI
        ,   'Medicare' AS CLAIMS_SOURCE 
        ,   COUNT(*) AS NUM_PROCEDURES
        ,   COUNT(DISTINCT C.DESY_SORT_KEY) AS NUM_PATIENTS 
    FROM    (
            SELECT  DISTINCT A.DESY_SORT_KEY 
                ,   A.CLM_THRU_DT 
                ,   B.HCPCS_CD AS PROCEDURE_CODE 
                ,   COALESCE(A.OP_PHYSN_NPI, A.AT_PHYSN_NPI) AS PHYSICIAN_NPI
            FROM    MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}{q} A 
            JOIN    MEDICARE_SAF_{yr}.dbo.out_revenuek_lds_100_{yr}{q} B 
            ON      A.CLAIM_NO = B.CLAIM_NO 
            WHERE   B.HCPCS_CD IN({cd})
            ) A
    JOIN    MEDICARE_SAF_{yr}.dbo.CLMS_DENOMINATOR_MEDICARE{q2} C 
    ON      A.DESY_SORT_KEY = C.DESY_SORT_KEY
    GROUP   BY PROCEDURE_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END 
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END 
        ,   A.PHYSICIAN_NPI
    """.format(yr=year, q=q, q2=q2, cd=cd)

    med_out_pr_df = pd.read_sql(med_out_pr_sql, ssms_conn)
    
    return med_out_pr_df

In [6]:
# for year in[2018, 2019]:
#     if year == 2019:
#         qtrly=1
#     else:
#         qtrly=0
#     print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
#     print('Running Medicare Outpatient for {yr}'.format(yr=year))
#     med_out_pr_result = pull_medicare_outpatient_procs(year, qtrly)
    
#     print('Medicare Outpatient for {yr} complete.'.format(yr=year))
#     dataframe = med_out_pr_result
    
#     print('Loading Medicare Outpatient for {yr} to BQ.'.format(yr=year))
#     tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_PROCEDURES_MEDICARE_YEARLY'
#     job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
#     print(job.result())
#     print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

In [8]:
yr = 2018
q = ''
qtrly = 0 

cpt_codes = pd.read_sql("""SELECT DISTINCT REV_CNTR_HCPCS_CD FROM MEDICARE_SAF_{yr}.dbo.OUTPATIENT_REV_CENTER{q} WITH(NOLOCK) WHERE REV_CNTR_HCPCS_CD IS NOT NULL""".format(yr=yr, q=q), ssms_conn)

cpt_codes = list(cpt_codes['REV_CNTR_HCPCS_CD'])

codes_in_table = bq_conn.query("""SELECT DISTINCT PROCEDURE_CODE FROM DS_WORK.ALL_PHYSICIAN_PROCEDURES_MEDICARE_YEARLY WHERE CLAIM_YEAR={yr}""".format(yr=yr)).result().to_dataframe()
codes_in_table = list(codes_in_table['PROCEDURE_CODE']) 

cpt_codes_new = ["""'"""+x+"""'""" for x in cpt_codes if x not in codes_in_table]

In [9]:
print(len(cpt_codes))
print(len(cpt_codes_new))

12354
12354


In [10]:
l  = list()
for z in range(0, len(cpt_codes_new)+1):
    if z % 100 == 0:
        l.append(z)

In [11]:
print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
print('Running Medicare Outpatient procedures for {yr}'.format(yr=yr))

for n in l: 
    code = ','.join(cpt_codes_new[n:n+100])
    med_out_pr_result = pull_medicare_outpatient_procs(year=yr, qtrly=qtrly, cd=code)
    dataframe = med_out_pr_result
    tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_PROCEDURES_MEDICARE_YEARLY'
    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    res = job.result()

print('Finished', datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/24/2020, 10:19:40
Running Medicare Outpatient procedures for 2018
Finished 06/24/2020, 14:30:30


In [6]:
yr = 2019
q = '_QTRLY'
qtrly = 1 

cpt_codes = pd.read_sql("""SELECT DISTINCT REV_CNTR_HCPCS_CD FROM MEDICARE_SAF_{yr}.dbo.OUTPATIENT_REV_CENTER{q} WITH(NOLOCK) WHERE REV_CNTR_HCPCS_CD IS NOT NULL""".format(yr=yr, q=q), ssms_conn)

cpt_codes = list(cpt_codes['REV_CNTR_HCPCS_CD'])

codes_in_table = bq_conn.query("""SELECT DISTINCT PROCEDURE_CODE FROM DS_WORK.ALL_PHYSICIAN_PROCEDURES_MEDICARE_YEARLY WHERE CLAIM_YEAR={yr}""".format(yr=yr)).result().to_dataframe()
codes_in_table = list(codes_in_table['PROCEDURE_CODE']) 

cpt_codes_new = ["""'"""+x+"""'""" for x in cpt_codes if x not in codes_in_table]

In [7]:
print(len(cpt_codes))
print(len(cpt_codes_new))

12206
4106


In [8]:
l  = list()
for z in range(0, len(cpt_codes_new)+1):
    if z % 100 == 0:
        l.append(z)

In [9]:
print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
print('Running Medicare Outpatient procedures for {yr}'.format(yr=yr))

for n in l: 
    code = ','.join(cpt_codes_new[n:n+100])
    med_out_pr_result = pull_medicare_outpatient_procs(year=yr, qtrly=qtrly, cd=code)
    dataframe = med_out_pr_result
    tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_PROCEDURES_MEDICARE_YEARLY'
    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    res = job.result()

print('Finished', datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/25/2020, 01:05:16
Running Medicare Outpatient procedures for 2019


/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'PROCEDURE_CODE'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Finished 06/25/2020, 02:57:26


#### Procedures - Commercial

In [5]:
def run_sql_BQ(filename):
    file = open(filename, 'r')
    file_text = file.read()
    file.close()
    
    sqlCommands = file_text.split(';')
    
    for i, command in enumerate(sqlCommands):
        bq_conn = bigquery.Client()
        try:
            job = bq_conn.query(command)
            print(job.result())
            print("Command successful: ", str(i))
        except:
            print("Command skipped: ", str(i))

In [6]:
# Commercial demo table prep
print(datetime.now())
run_sql_BQ('Commercial_Prep.txt')
print(datetime.now())

2020-06-29 10:09:23.523477
Command successful:  0
Command successful:  1
2020-06-29 10:14:37.122184


In [7]:
# Commercial Procedures - update table 
print(datetime.now())
run_sql_BQ('CLAIMS_BY_CODE_px_table_COMM.txt')
print(datetime.now())

2020-06-29 10:14:37.127109
Command successful:  0
Command successful:  1
2020-06-29 10:32:47.452087


In [8]:
### COMMERCIAL ICD ###

def pull_commercial_icd_procs(year):
    
    comm_icd_sql = """
        INSERT INTO DS_WORK.ALL_PHYSICIAN_PROCEDURES_COMMERCIAL_YEARLY (CLAIM_YEAR, PROCEDURE_CODE, AGE_RANGE, GENDER, PHYSICIAN_NPI, CLAIMS_SOURCE, NUM_PROCEDURES, NUM_PATIENTS)
        SELECT CLAIM_YEAR
            ,  PROCEDURE_CODE
            ,  CASE WHEN AGE < 45 THEN 'Less than 45'
                    WHEN AGE >=45 AND AGE <55 THEN '45-54'
                    WHEN AGE>=55 AND AGE<65 THEN '55-64'
                    WHEN AGE>=65 AND AGE<75 THEN '65-74'
                    WHEN AGE>=75 AND AGE<85 THEN '75-84'
                    WHEN AGE>=85 THEN '85+'
               ELSE 'Unknown' END AS AGE_RANGE
            ,  GENDER
            ,  PHYSICIAN_NPI
            ,  'Commercial' AS CLAIMS_SOURCE
            ,  COUNT(*) AS NUM_PROCEDURES 
            ,  COUNT(DISTINCT DPID) AS NUM_PATIENTS
          FROM (
                SELECT DISTINCT DPID
                    ,  RENDERING_DEFHC_ID AS PHYSICIAN_NPI
                    ,  GENDER
                    ,  CAST(YEAR_OF_BIRTH AS INT64) AS YEAR_OF_BIRTH
                    ,  CLAIM_YEAR - CAST(YEAR_OF_BIRTH AS INT64) AS AGE
                    ,  CLAIM_YEAR
                    ,  SAFE_CAST(CONCAT(SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 1, 4), '-', SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 5, 2), '-', SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 7, 2)) AS DATE) AS SERVICE_TO_DATE
                    ,  ProcedureCode AS PROCEDURE_CODE
                FROM   DS_WORK.CLAIMS_BY_CODE_PX_SOP A 
                JOIN   CommercialHealthClaims.PhysicianLookup B
                ON     A.RENDERING_DEFHC_ID = B.NPI 
                WHERE  CLAIM_YEAR = {yr}
                AND    ICD_HCPCS = 1
                AND    SourceOfPaymentCode NOT IN ('MA','MB')
                ) A
         GROUP BY CLAIM_YEAR
            ,  PROCEDURE_CODE
            ,  CASE WHEN AGE < 45 THEN 'Less than 45'
                    WHEN AGE >=45 AND AGE <55 THEN '45-54'
                    WHEN AGE>=55 AND AGE<65 THEN '55-64'
                    WHEN AGE>=65 AND AGE<75 THEN '65-74'
                    WHEN AGE>=75 AND AGE<85 THEN '75-84'
                    WHEN AGE>=85 THEN '85+'
               ELSE 'Unknown' END
            ,  GENDER
            ,  PHYSICIAN_NPI
""".format(yr=year)
    
    comm_icd = bq_conn.query(comm_icd_sql)
    comm_icd.result()

In [9]:
for year in[2018, 2019]:
    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    print('Running Commercial ICD Procs {}'.format(year))
    comm_pr_icd_result = pull_commercial_icd_procs(year)

    print('Commercial ICD procs complete {}.'.format(year))
    dataframe = comm_pr_icd_result

    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/29/2020, 10:32:47
Running Commercial ICD Procs 2018
Commercial ICD procs complete 2018.
06/29/2020, 10:33:28
06/29/2020, 10:33:28
Running Commercial ICD Procs 2019
Commercial ICD procs complete 2019.
06/29/2020, 10:34:24


In [10]:
### COMMERCIAL HCPCS ###

def pull_commercial_hcpcs_procs(year):
    
    comm_hcpcs_sql = """
        INSERT INTO DS_WORK.ALL_PHYSICIAN_PROCEDURES_COMMERCIAL_YEARLY (CLAIM_YEAR, PROCEDURE_CODE, AGE_RANGE, GENDER, PHYSICIAN_NPI, CLAIMS_SOURCE, NUM_PROCEDURES, NUM_PATIENTS)
        SELECT CLAIM_YEAR
            ,  PROCEDURE_CODE
            ,  CASE WHEN AGE < 45 THEN 'Less than 45'
                    WHEN AGE >=45 AND AGE <55 THEN '45-54'
                    WHEN AGE>=55 AND AGE<65 THEN '55-64'
                    WHEN AGE>=65 AND AGE<75 THEN '65-74'
                    WHEN AGE>=75 AND AGE<85 THEN '75-84'
                    WHEN AGE>=85 THEN '85+'
               ELSE 'Unknown' END AS AGE_RANGE
            ,  GENDER
            ,  PHYSICIAN_NPI
            ,  'Commercial' AS CLAIMS_SOURCE
            ,  COUNT(*) AS NUM_PROCEDURES 
            ,  COUNT(DISTINCT DPID) AS NUM_PATIENTS
          FROM (
                SELECT DISTINCT DPID
                    ,  RENDERING_DEFHC_ID AS PHYSICIAN_NPI
                    ,  GENDER
                    ,  CAST(YEAR_OF_BIRTH AS INT64) AS YEAR_OF_BIRTH
                    ,  CLAIM_YEAR - CAST(YEAR_OF_BIRTH AS INT64) AS AGE
                    ,  CLAIM_YEAR
                    ,  SAFE_CAST(CONCAT(SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 1, 4), '-', SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 5, 2), '-', SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 7, 2)) AS DATE) AS SERVICE_TO_DATE
                    ,  ProcedureCode AS PROCEDURE_CODE
                FROM   DS_WORK.CLAIMS_BY_CODE_PX_SOP A 
                JOIN   CommercialHealthClaims.PhysicianLookup B
                ON     A.RENDERING_DEFHC_ID = B.NPI 
                WHERE  CLAIM_YEAR = {yr}
                AND    ICD_HCPCS = 2
                AND    SourceOfPaymentCode NOT IN ('MA','MB')
                ) A
         GROUP BY CLAIM_YEAR
            ,  PROCEDURE_CODE
            ,  CASE WHEN AGE < 45 THEN 'Less than 45'
                    WHEN AGE >=45 AND AGE <55 THEN '45-54'
                    WHEN AGE>=55 AND AGE<65 THEN '55-64'
                    WHEN AGE>=65 AND AGE<75 THEN '65-74'
                    WHEN AGE>=75 AND AGE<85 THEN '75-84'
                    WHEN AGE>=85 THEN '85+'
               ELSE 'Unknown' END
            ,  GENDER
            ,  PHYSICIAN_NPI
""".format(yr=year)
    
    comm_hcpcs = bq_conn.query(comm_hcpcs_sql)
    comm_hcpcs.result()


In [11]:
for year in[2018, 2019]:
    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    print('Running Commercial HCPCS Procs {}'.format(year))
    comm_pr_hcpcs_result = pull_commercial_hcpcs_procs(year)

    print('Commercial HCPCS procs complete {}.'.format(year))
    dataframe = comm_pr_hcpcs_result

    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/29/2020, 10:34:24
Running Commercial HCPCS Procs 2018
Commercial HCPCS procs complete 2018.
06/29/2020, 10:36:30
06/29/2020, 10:36:30
Running Commercial HCPCS Procs 2019
Commercial HCPCS procs complete 2019.
06/29/2020, 10:38:55


#### Diagnoses - Medicare

In [12]:
## MEDICARE INPATIENT DIAGNOSES ##

def pull_medicare_inpatient_dx(year, qtrly=0):
    
    if qtrly == 1:
        q = '_q'
        q2 = '_QTRLY'
    else:
        q = ''
        q2 = ''
    
    med_inp_dx_sql = """
    SELECT  {yr} AS CLAIM_YEAR
        ,   DIAGNOSIS_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END AS AGE_RANGE 
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END AS GENDER 
        ,   A.PHYSICIAN_NPI
        ,   'Medicare Inpatient' AS CLAIMS_SOURCE 
        ,   COUNT(*) AS NUM_DIAGNOSES
        ,   COUNT(DISTINCT C.DESY_SORT_KEY) AS NUM_PATIENTS 
    FROM    (
            SELECT  DISTINCT A.DESY_SORT_KEY 
                ,   A.CLM_THRU_DT 
                ,   B.CLM_DGNS_CD_ORIG AS DIAGNOSIS_CODE 
                ,   COALESCE(A.OP_PHYSN_NPI, A.AT_PHYSN_NPI) AS PHYSICIAN_NPI
            FROM    MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}{q} A 
            JOIN    MEDICARE_SAF_{yr}.dbo.INPATIENT_CLM_DGNS{q2} B 
            ON      A.CLAIM_NO = B.CLAIM_NO 
            ) A
    JOIN    MEDICARE_SAF_{yr}.dbo.CLMS_DENOMINATOR_MEDICARE{q2} C 
    ON      A.DESY_SORT_KEY = C.DESY_SORT_KEY
    GROUP   BY  DIAGNOSIS_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END
        ,   A.PHYSICIAN_NPI
    """.format(yr=year, q=q, q2=q2)

    med_inp_dx_df = pd.read_sql(med_inp_dx_sql, ssms_conn)
    
    return med_inp_dx_df 


In [13]:
for year in[2018, 2019]:
    if year == 2019:
        qtrly=1
    else:
        qtrly=0
    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    print('Running Medicare Inpatient diagnoses for {yr}'.format(yr=year))
    med_inp_dx_result = pull_medicare_inpatient_dx(year, qtrly)
    
    print('Medicare Inpatient for {yr} complete.'.format(yr=year))
    dataframe = med_inp_dx_result
    
    print('Loading Medicare Inpatient diagnoses for {yr} to BQ.'.format(yr=year))
    tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_DIAGNOSES_MEDICARE_YEARLY'
    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    print(job.result())
    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/29/2020, 10:53:59
Running Medicare Inpatient diagnoses for 2018
Medicare Inpatient for 2018 complete.
Loading Medicare Inpatient diagnoses for 2018 to BQ.


/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'DIAGNOSIS_CODE'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


06/29/2020, 11:32:46
06/29/2020, 11:32:46
Running Medicare Inpatient diagnoses for 2019
Medicare Inpatient for 2019 complete.
Loading Medicare Inpatient diagnoses for 2019 to BQ.
06/29/2020, 12:18:19


In [5]:
## MEDICARE OUTPATIENT DIAGNOSES ##

def pull_medicare_outpatient_dx(year, qtrly, code):
    
    if qtrly == 1:
        q = '_q'
        q2 = '_QTRLY'
    else:
        q = ''
        q2 = ''
        
    med_out_dx_sql = """
    SELECT   {yr} AS CLAIM_YEAR
        ,   DIAGNOSIS_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END AS AGE_RANGE 
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END AS GENDER 
        ,   A.PHYSICIAN_NPI
        ,   'Medicare Outpatient' AS CLAIMS_SOURCE 
        ,   COUNT(*) AS NUM_DIAGNOSES
        ,   COUNT(DISTINCT C.DESY_SORT_KEY) AS NUM_PATIENTS 
    FROM    (
            SELECT  DISTINCT A.DESY_SORT_KEY 
                ,   A.CLM_THRU_DT 
                ,   B.CLM_DGNS_CD_ORIG AS DIAGNOSIS_CODE 
                ,   COALESCE(A.OP_PHYSN_NPI, A.AT_PHYSN_NPI) AS PHYSICIAN_NPI
            FROM    MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}{q} A 
            JOIN    MEDICARE_SAF_{yr}.dbo.OUTPATIENT_CLM_DGNS{q2} B 
            ON      A.CLAIM_NO = B.CLAIM_NO 
            WHERE   B.CLM_DGNS_CD_ORIG in({cd})
            ) A
    JOIN    MEDICARE_SAF_{yr}.dbo.CLMS_DENOMINATOR_MEDICARE{q2} C 
    ON      A.DESY_SORT_KEY = C.DESY_SORT_KEY
    GROUP   BY  DIAGNOSIS_CODE
        ,   CASE WHEN C.AGE < 45 THEN 'Less than 45'
                 WHEN C.AGE>=45 AND C.AGE<55 THEN '45-54'
                 WHEN C.AGE>=55 AND C.AGE<65 THEN '55-64'
                 WHEN C.AGE>=65 AND C.AGE<75 THEN '65-74'
                 WHEN C.AGE>=75 AND C.AGE<85 THEN '75-84'
                 WHEN C.AGE>=85 THEN '85+'
            ELSE 'Unknown' END 
        ,   CASE WHEN SEX_CODE = 1 THEN 'M' WHEN SEX_CODE = 2 THEN 'F' ELSE 'U' END 
        ,   A.PHYSICIAN_NPI
    """.format(yr=year, q=q, q2=q2, cd=code)

    med_out_dx_df = pd.read_sql(med_out_dx_sql, ssms_conn)
    
    return med_out_dx_df

In [6]:
year = 2018
qtrly = 0
q2 = ''

dx_sql = """SELECT DISTINCT CLM_DGNS_CD_ORIG AS DIAGNOSIS_CODE FROM MEDICARE_SAF_{yr}.dbo.OUTPATIENT_CLM_DGNS{q2}""".format(yr=year,q2=q2)
dx_df = pd.read_sql(dx_sql, ssms_conn)
dx_cds = list(dx_df['DIAGNOSIS_CODE'].unique())

In [7]:
dx_excl_sql = """SELECT DISTINCT DIAGNOSIS_CODE FROM DS_WORK.ALL_PHYSICIAN_DIAGNOSES_MEDICARE_YEARLY WHERE CLAIMS_SOURCE = 'Medicare Outpatient' AND CLAIM_YEAR={yr}""".format(yr=year)

In [8]:
dx_excl_df = bq_conn.query(dx_excl_sql).result().to_dataframe()

In [9]:
dx_excl = list(dx_excl_df['DIAGNOSIS_CODE'].unique())

In [10]:
dx_cds_query = ["""'"""+x+"""'""" for x in dx_cds if x not in dx_excl]

In [11]:
l  = list()
for z in range(0, len(dx_cds_query)+1):
    if z % 50 == 0:
        l.append(z)

In [12]:
len(dx_cds_query)

39042

In [13]:
print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
print('Running Medicare Outpatient diagnoses for {yr}'.format(yr=year))

for n in l: 
    code = ','.join(dx_cds_query[n:n+50])
    med_out_dx_result = pull_medicare_outpatient_dx(year, qtrly, code)
    dataframe = med_out_dx_result
    tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_DIAGNOSES_MEDICARE_YEARLY'
    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    res = job.result()

print('Finished', datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/25/2020, 09:00:53
Running Medicare Outpatient diagnoses for 2018


/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'DIAGNOSIS_CODE'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Finished 06/25/2020, 17:30:59


In [6]:
year = 2019
qtrly = 1
q2 = '_QTRLY'

dx_sql = """SELECT DISTINCT CLM_DGNS_CD_ORIG AS DIAGNOSIS_CODE FROM MEDICARE_SAF_{yr}.dbo.OUTPATIENT_CLM_DGNS{q2}""".format(yr=year,q2=q2)
dx_df = pd.read_sql(dx_sql, ssms_conn)
dx_cds = list(dx_df['DIAGNOSIS_CODE'].unique())

dx_excl_sql = """SELECT DISTINCT DIAGNOSIS_CODE FROM DS_WORK.ALL_PHYSICIAN_DIAGNOSES_MEDICARE_YEARLY WHERE CLAIMS_SOURCE = 'Medicare Outpatient' AND CLAIM_YEAR={yr}""".format(yr=year)
dx_excl_df = bq_conn.query(dx_excl_sql).result().to_dataframe()
dx_excl = list(dx_excl_df['DIAGNOSIS_CODE'].unique())

In [7]:
dx_cds_query = ["""'"""+x+"""'""" for x in dx_cds if x not in dx_excl]

In [8]:
len(dx_cds_query)

37761

In [9]:
l  = list()
for z in range(0, len(dx_cds_query)+1):
    if z % 50 == 0:
        l.append(z)

In [10]:
print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
print('Running Medicare Outpatient diagnoses for {yr}'.format(yr=year))

for n in l: 
    code = ','.join(dx_cds_query[n:n+50])
    med_out_dx_result = pull_medicare_outpatient_dx(year, qtrly, code)
    dataframe = med_out_dx_result
    tablename = 'stable-healer-231019.DS_WORK.ALL_PHYSICIAN_DIAGNOSES_MEDICARE_YEARLY'
    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    res = job.result()

print('Finished', datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/26/2020, 13:52:53
Running Medicare Outpatient diagnoses for 2019


/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'DIAGNOSIS_CODE'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Finished 06/26/2020, 20:16:57


In [14]:
# Commercial diagnoses
run_sql_BQ('CLAIMS_BY_CODE_dx_table_COMM.txt')

Command successful:  0
Command successful:  1
Command skipped:  2


In [17]:
## COMMERCIAL DIAGNOSES ## 

def pull_commercial_dx(year):
    
    comm_dx_sql = """
        INSERT INTO DS_WORK.ALL_PHYSICIAN_DIAGNOSES_COMMERCIAL_YEARLY (CLAIM_YEAR, DIAGNOSIS_CODE, AGE_RANGE, GENDER, PHYSICIAN_NPI, CLAIMS_SOURCE, NUM_DIAGNOSES, NUM_PATIENTS)
        SELECT CLAIM_YEAR
            ,  DIAGNOSIS_CODE
            ,  CASE WHEN AGE < 45 THEN 'Less than 45'
                    WHEN AGE >=45 AND AGE <55 THEN '45-54'
                    WHEN AGE>=55 AND AGE<65 THEN '55-64'
                    WHEN AGE>=65 AND AGE<75 THEN '65-74'
                    WHEN AGE>=75 AND AGE<85 THEN '75-84'
                    WHEN AGE>=85 THEN '85+'
               ELSE 'Unknown' END AS AGE_RANGE
            ,  GENDER
            ,  PHYSICIAN_NPI
            ,  'Commercial' AS CLAIMS_SOURCE
            ,  COUNT(*) AS NUM_DIAGNOSES 
            ,  COUNT(DISTINCT DPID) AS NUM_PATIENTS
          FROM (
                SELECT DISTINCT DPID
                    ,  RENDERING_DEFHC_ID AS PHYSICIAN_NPI
                    ,  GENDER
                    ,  CAST(YEAR_OF_BIRTH AS INT64) AS YEAR_OF_BIRTH
                    ,  CLAIM_YEAR - CAST(YEAR_OF_BIRTH AS INT64) AS AGE
                    ,  CLAIM_YEAR
                    ,  SAFE_CAST(CONCAT(SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 1, 4), '-', SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 5, 2), '-', SUBSTR(CAST(SERVICE_TO_DATE AS STRING), 7, 2)) AS DATE) AS SERVICE_TO_DATE
                    ,  DiagnosisCode AS DIAGNOSIS_CODE
                FROM   DS_WORK.CLAIMS_BY_CODE_DX_SOP A 
                JOIN   CommercialHealthClaims.PhysicianLookup B
                ON     A.RENDERING_DEFHC_ID = B.NPI 
                WHERE  CLAIM_YEAR = {yr}
                AND    SourceOfPaymentCode NOT IN ('MA','MB')
                ) A
         GROUP BY CLAIM_YEAR
            ,  DIAGNOSIS_CODE
            ,  CASE WHEN AGE < 45 THEN 'Less than 45'
                    WHEN AGE >=45 AND AGE <55 THEN '45-54'
                    WHEN AGE>=55 AND AGE<65 THEN '55-64'
                    WHEN AGE>=65 AND AGE<75 THEN '65-74'
                    WHEN AGE>=75 AND AGE<85 THEN '75-84'
                    WHEN AGE>=85 THEN '85+'
               ELSE 'Unknown' END
            ,  GENDER
            ,  PHYSICIAN_NPI
""".format(yr=year)
    
    comm_dx = bq_conn.query(comm_dx_sql)
    comm_dx.result()

In [18]:
for year in[2018, 2019]:
    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    print('Running Commercial ICD Diagnoses {}'.format(year))
    comm_dx_icd_result = pull_commercial_dx(year)

    print('Commercial ICD diagnoses complete {}.'.format(year))
    dataframe = comm_dx_icd_result

    print(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

06/29/2020, 12:51:10
Running Commercial ICD Diagnoses 2018
Commercial ICD diagnoses complete 2018.
06/29/2020, 12:53:52
06/29/2020, 12:53:52
Running Commercial ICD Diagnoses 2019
Commercial ICD diagnoses complete 2019.
06/29/2020, 12:57:03
